In [1]:
import re
import pandas as pd
import requests
import pdfplumber

In [2]:
arquivo = pdfplumber.open("arquivo/Extrato Banco.pdf")

In [3]:
with pdfplumber.open("arquivo/Extrato Agrupado Simplificado.pdf") as pdf:
    a = pdf.pages[0]
    a.chars[0]

In [4]:
b = a.extract_text().split("\n")

In [5]:
serie = pd.Series(b)

In [6]:
contratos = []
aluguel = []

In [7]:
for x in range (0, len(serie)):
    linha = serie.loc[x]
    primeira_palavra = linha.split(" ")[0]
    
    if primeira_palavra == "Emitido":
        data_emicao = linha.split(" ")[2]
        
    if primeira_palavra == "Contrato":
        contratos.append(linha[len("Contrato "):])
        
    if primeira_palavra == "Aluguel":
        aluguel.append(float((linha[len("Aluguel - 01/12/2020 até 31/12/2020 "):].replace(",", "."))))

In [8]:
aluguel

[368.3, 368.3, 429.68, 437.77]

In [10]:
dir(pdfplumber)

['PDF',
 '__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '__version__',
 '_version',
 'container',
 'convert',
 'load',
 'open',
 'page',
 'pdf',
 'pdfminer',
 'set_debug',
 'sys',
 'table',
 'utils']

In [14]:
dir(pdfplumber.page)

['ALL_ATTRS',
 'Container',
 'CroppedPage',
 'DECIMAL_ATTRS',
 'DerivedPage',
 'FilteredPage',
 'Page',
 'TableFinder',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'lt_pat',
 're',
 'resolve_all',
 'test_proposed_bbox',
 'utils']

<h1>Desafio 1</h1>

In [17]:
with pdfplumber.open("arquivo/Rua Ângelo Martins, 445 1-10.pdf") as pdf:
    RAM = pdf.pages[0]
    RAM.chars[0]

In [20]:
text_RAM = RAM.extract_text().split("\n")

In [21]:
text_RAM

['SEMEC - PREFEITURA MUNICIPAL DE MACEIO',
 'SECRETARIA MUNICIPAL DE ECONOMIA - SEMEC',
 'DOCUMENTO DE ARRECADAÇÃO MUNICIPAL - D.A.M MENSAL Text',
 'Perdeu o prazo do DAM? Emita automaticamente através do QRCODE',
 'Contribuinte D.A.M.',
 'MANOEL CORREIA TELLES 66114110 / 2021',
 'CPF/CNPJ Inscrição / C.M.C. Tributos Referência Vencimento Nosso Número',
 '002.643.184-04 000000000120433 I.P.T.U. 2021 1/10 31/03/2021 11000000075755253',
 'Endereço de Localização',
 'Logradouro: RUA - ÂNGELO MARTINS 00445 Cep: 57030592',
 'Número:',
 'Bairro: PONTA DA TERRA Complemento: Lote: 0530 Quadra: 0102',
 'Loteamento: Quadra Lot.: Lote Lot.:',
 'Imóvel: PREDIAL Identificação: 05010240530001',
 'Área Terreno: 119,00 Área Total Construída: 73,58 Área Construída Unidade: 73,58',
 'Valor Venal Terreno: 24.273,80 Valor Venal Construção: 51.307,41',
 'Valor Venal Imóvel: 75.581,21 Base de Cálculo: 45.348,73 Alíquota:  1,0000',
 'IPTU PREDIAL - PRINCIPAL 45,35',
 'COLETA DE LIXO 25,52 Valor R$ 70,87',
 '

In [36]:
for x in range (0, len(text_RAM)):
    if text_RAM[x][0:len('Total a Pagar: ')] == 'Total a Pagar: ':
        total_a_pagar = float(text_RAM[x][len('Total a Pagar: '):].replace(',', '.'))

In [37]:
total_a_pagar

70.87

In [22]:
codBarras = text_RAM[-1]

In [23]:
codBarras

'81620000000-7 70872485202-3 10331110000-4 00075755253-4'